# Model 1: rainfall on DEM

With this notebook you will build a simple H2Flow project; rain on a digital elevation model (DEM). All other H2Flow parameters (e.g. friction, infiltration, etc.) we will keep constant.

Objective: to get familiar with H2Flow grid, and minimal project structure

Steps in this notebook:
1. Prepare input data:
    - Process the HydroDEM into H2Flow ASCI dem
    - Process ASCII rainfall to a H2Flow timeseries
2. Write settings:
    - Settings for H2Flow grid generator (input_grid_gen)
    - Input-settings for your model-run (input)
    - Output-settings for your model-run (settings_output)
    - Numerical settings for the computational core (settings_omp)
3. Write IO root

In [1]:
from pathlib import Path, PurePosixPath
import geopandas as gpd
import pandas as pd
import rasterio
import copy

## 1. Prepare input data:

Prepare the input for this notebook by:
- downloading your data from: https://drive.google.com/drive/folders/1e3kjUWD-ikHo_TSmjCY86QyL-40R9xt3?usp=sharing.
- putting the data into ..\data\de_tol relative to the path-location of this notebook.

Below the input paths are defined, they should all exist

In [2]:
# Repos data path
data_path = Path(r"../data")

# level 1 source-data
source_path = data_path.joinpath("de_tol")
dem_tif = source_path.joinpath("dem_kockengen_20160905.tif")
wla_gpkg = source_path.joinpath("waterlevel_areas.gpkg")
gauges_gpkg = source_path.joinpath("gauges.gpkg")
rainfall_path = source_path.joinpath("rainfall")

# level 2 project directory
project_dir = data_path.joinpath("case")
project_dir.mkdir(parents=True, exist_ok=True)

# h2flow run_directory
run_dir = data_path.joinpath("exe")
run_dir.mkdir(parents=True, exist_ok=True)

### Process the HydroDEM into H2Flow ASCI dem

You read the DEM into memory and read the (meta-)data. We save the DEM as ESRI ASCII file: https://desktop.arcgis.com/en/arcmap/10.3/manage-data/raster-and-images/esri-ascii-raster-format.html

You specify one variable:
- <b>dem_version:</b> version number of the dem-file

In [3]:
dem_version = 1

# read DEM into memory and save it as ESRI ASCI-grid
with rasterio.open(dem_tif) as src:
    profile = src.profile
    data = src.read(1)
    dem_res = copy.copy(src.res[0])

profile["driver"] = "AAIGrid"

print(profile)

dem_file = f'depthasci,imax= {profile["width"]},jmax= {profile["height"]},netw1d=  0.  {dem_version}'

with rasterio.open(project_dir.joinpath(dem_file), "w+", **profile) as dst:
    dst.write(data, 1)

{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 10274, 'height': 12146, 'count': 1, 'crs': CRS.from_epsg(28992), 'transform': Affine(0.5, 0.0, 124434.5,
       0.0, -0.5, 464846.0), 'tiled': False, 'compress': 'deflate', 'interleave': 'band'}


### Process ASCII rainfall to a H2Flow timeseries

We have rainfall time-series in ESRI-ASCI format, but we will supply the H2Flow model with a CSV containing one time-series. For that we take the following steps per ASCI-file:
1. Compute the timestep of the asci-file
2. Sample the rainfall value from the asci-file
3. Append timestep and value to DataFrame

The result you save in a CSV-file that contains all data. This CSV has the following header information:<br>
<i>{rainfall_version} {m_conversion} {t_conversion}<br>
{number_gauges} {bounds}</i>

In this process you'll need the following variables:
- <b>rainfall_version:</b> version of the rainfall event
- <b>m_conversion:</b> conversion from input unit (mm/h) to H2Flow unit (m/s)
- <b>t_conversion:</b> conversion form input unit (mm/h) to H2Flow unit (m/s)
- <b>number_gauges:</b> amount of gauges used

In [4]:
# input variables
rainfall_version = -1
m_conversion = 0.001
t_conversion = -3600.0
number_gauges = 1

# computed variables
bounds = " ".join([str(int(i)) for i in list(src.bounds)]) # boundary of rainfall
rainfall_ascs = rainfall_path.glob("*.ASC") # input rainfall asci's

gdf = gpd.read_file(gauges_gpkg) # gauges-file (contains 1 gauge for now)
xy = [(geom.x, geom.y) for geom in gdf["geometry"]] # xy-tuple from rainfall gauge

columns = {"days": int, "hours": str, "minutes": str, "seconds": int, "value": float}
df = pd.DataFrame(columns=columns)

# iteration over rainfall asci's
for idx, asc in enumerate(rainfall_ascs):
    datetime = pd.to_datetime(asc.stem, format="NSL_%Y%m%d_%H")

    if idx == 0:
        ref_datetime = copy.copy(datetime)

    time_delta = datetime - ref_datetime

    days = time_delta.components.days
    hours = f"{time_delta.components.hours:02d}"
    minutes = f"{time_delta.components.minutes:02d}"
    seconds = time_delta.components.seconds

    with rasterio.open(asc) as src:
        value = [val[0] for val in src.sample(xy)][0]
        df.loc[idx] = [days, hours, minutes, seconds, value]

# writing output to CSV
rain_path = project_dir.joinpath(f"rain. {rainfall_version}")

## write header
header = f"""{rainfall_version} {m_conversion} {t_conversion}
{number_gauges} {bounds}
"""
rain_path.write_text(header)

## append dataframe
df.astype(columns).to_csv(
    rain_path, mode="a", index=False, header=False, float_format="%.2f"
)

print(rain_path.read_text())

C:\Anaconda3\envs\work\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


-1 0.001 -3600.0
1 124434 458773 129571 464846
0,00,00,0,0.00
0,01,00,0,0.00
0,02,00,0,17.23
0,03,00,0,1.27
0,04,00,0,4.02
0,05,00,0,0.00
0,06,00,0,0.00
0,07,00,0,0.00
0,08,00,0,15.29
0,09,00,0,0.00
0,10,00,0,0.34
0,11,00,0,17.56
0,12,00,0,39.75
0,13,00,0,33.85
0,14,00,0,0.16
0,15,00,0,0.05
0,16,00,0,1.55
0,17,00,0,0.04
0,18,00,0,0.00
0,19,00,0,0.00
0,20,00,0,0.00
0,21,00,0,0.00
0,22,00,0,0.00
0,23,00,0,0.00
1,00,00,0,0.00



## 2. Write settings

### Settings for H2Flow grid generator (input_grid_gen)

Below the settings that will be used by the grid-generator. The grid-generator will generate a H2Flow model grid using the DEM and other parameters. In the cell below we will create the settings-file for the grid-generator

We define the following input parameters:
- <b>grid_version:</b> version of the H2Flow grid to be generated
- <b>quadtree_levels:</b> We will not generate a quadtree here, so keep it at 1
- <b>finest_grid_size:</b> The smallest grid-size in units of cell-size of the original DEM, we'll use the value of 20
- <b>manning_value:</b> we will use a constant-value of 0.04 
- <b>infiltration_value:</b> we will use no infiltration here 0
- <b>initial_water_level:</b> we will use a constant value of -1.8

In [5]:
grid_version = 1
quadtree_levels = 1
finest_grid_size = 20
manning_value = 0.04
infiltration_value = 0
initial_water_level = -1.8

# generate file path
grid_gen_file = f"input_grid_gen.  {grid_version}"
grid_gen_path = project_dir.joinpath(grid_gen_file)

# generate file string
grid_gen_settings = f"""{grid_version} {dem_version} 0 0 !grid number, depthfile, 1D network, Ridges number
{quadtree_levels} !quadtree max. level
{finest_grid_size} !finest coarse grid resolution (how many dem pixels do you lump together?)
{profile["width"]} !ncols
{profile["height"]} !nrows
0.25 0.25 1.0 !table_step_size,subgrid params,1d spacing
{manning_value} 1 !manning value 
{infiltration_value} 1.0 20.0 !infiltration parameters - inactive with 0
{initial_water_level} 0 !initial water level default and file for s00, 0 means no files
"""

# write
grid_gen_path.write_text(grid_gen_settings)

# print
print(grid_gen_settings)

1 1 0 0 !grid number, depthfile, 1D network, Ridges number
1 !quadtree max. level
20 !finest coarse grid resolution (how many dem pixels do you lump together?)
10274 !ncols
12146 !nrows
0.25 0.25 1.0 !table_step_size,subgrid params,1d spacing
0.04 1 !manning value 
0 1.0 20.0 !infiltration parameters - inactive with 0
-1.8 0 !initial water level default and file for s00, 0 means no files



### Input-settings for your model-run (input)

In the cell below we will write the settings that will be used by H2Flow core for this specific model run. We define the following variables:
- <b>run_number:</b> run-version-number
- <b>advection_2d:</b> 0 or 1. This is an important setting as it will switch on/off advection in the 2d domain
- <b>advection_1d:</b> 0 or 1. This is an important setting as it will switch on/off advection in the 1d domain
- <b>timestep_secs:</b> Your timestep in seconds. This is important in combination with the advection switch. A too large timestep in combination with advection will cause instability

The rest we have defined above

In [6]:
run_number = 1
advection_2d = 0
advection_1d = 0
timestep_secs = 10

# generate file path
input_settings_file = f"input.  {run_number}"
input_settings_path = project_dir.joinpath(input_settings_file)

# generate file string
input_settings=f"""{grid_version} !grid number
0 !network number
{quadtree_levels} !highest level of quadtree
{finest_grid_size} !finest grid spacing number of dem cells to lump
{profile["width"]} !ncols
{profile["height"]} !nrows
{advection_2d} {advection_1d} !advection2d switch, advection1d switch
{float(timestep_secs)}d0 !dt #settings: time_step
0 {time_delta.components.days} {time_delta.components.hours} {time_delta.components.minutes} {time_delta.components.seconds} !total time of simulation  mon :: days :: hours :: mins :: seconds
0 0 !number of 2d boundaries, number of 1d boundaries (skip if none), number of for boundary data file extension 
1 !npc,number of prediction correction steps for flooding default
{initial_water_level} !initial waterlevel > 10^7 directs read from file sini_ generated from s00, otherwise water level applied everywhere
{rainfall_version} !rain -> read from file if negative file extension number
0  0.0d0 0.0d0 0.0d0 3600.0d0 0.0d0 !infiltration 2d parameters
1.0d-5 !wetting threshold
0 !number of pump locations
"""
# write
input_settings_path.write_text(input_settings)

# print
print(input_settings)

1 !grid number
0 !network number
1 !highest level of quadtree
20 !finest grid spacing number of dem cells to lump
10274 !ncols
12146 !nrows
0 0 !advection2d switch, advection1d switch
10.0d0 !dt #settings: time_step
0 1 0 0 0 !total time of simulation  mon :: days :: hours :: mins :: seconds
0 0 !number of 2d boundaries, number of 1d boundaries (skip if none), number of for boundary data file extension 
1 !npc,number of prediction correction steps for flooding default
-1.8 !initial waterlevel > 10^7 directs read from file sini_ generated from s00, otherwise water level applied everywhere
-1 !rain -> read from file if negative file extension number
0  0.0d0 0.0d0 0.0d0 3600.0d0 0.0d0 !infiltration 2d parameters
1.0d-5 !wetting threshold
0 !number of pump locations



### Output-settings for your model-run (settings_output)

In the cell below we define the output-settings used by the H2Flow core for this specific model run. We define the following variables:
- <b>output_version:</b> output-version-number
- <b>point_observations:</b> a list of tuples with (x,y) observation points
- <b>horizontal_line_observations</b> a list with lines on the horizontal cell interfaces defined by start and end point [[(xstart,ystart) (xend, yend)]]
- <b>vertical_line_observations</b> a list with lines on the vertical cell interfaces defined by start and end point [[(xstart,ystart) (xend, yend)]]

In [7]:
output_version = 1

# these can be later processed from feature-files
point_observations = [(125023.549, 463079.690), (127761.300, 463951.612)]
horizontal_line_observations = [[(127831.50, 464373.50), (127975.50, 464373.50)]]
vertical_line_observations = [[(127369.50, 464423.50), (127369.50, 464539.50)]]

# convert observation feature-maps to strings
point_observations_str = "\n".join([" ".join(map(str, i)) for i in point_observations])
horizontal_line_observations_str = "\n".join([" ".join([" ".join(map(str, p)) for p in i]) for i in horizontal_line_observations])
vertical_line_observations_str = "\n".join([" ".join([" ".join(map(str, p)) for p in i]) for i in vertical_line_observations])

# generate file path
output_settings_file = f"settings_output.  {output_version}"
output_settings_path = project_dir.joinpath(output_settings_file)

# generate file string
output_settings=f"""{ref_datetime.strftime("%Y %m %d %H %M %S")} !start date
{bounds} !bounding box
{profile["crs"].to_epsg()} !epsg
{dem_res} !dem resolution
1 30 300 !logging display interval, interval in timesteps for 1d plots, interval in timesteps for 2d plots
0.01 0.002 !ponding water level, velocity threshold for 2d plots
0.05 0.001 !flowthreshold for 2d plots, hazard thresholds
{len(point_observations)} !number of water levels observation locations
{point_observations_str}
{len(horizontal_line_observations)} !number of horizontal cross-sections for discharge
{horizontal_line_observations_str} !horizontal line points left to right
{len(vertical_line_observations)} !number of vertical cross-sections for discharge
{vertical_line_observations_str} !vertical line points bottom to top
"""

# write
output_settings_path.write_text(output_settings)

# print
print(output_settings)

2014 07 27 21 00 00 !start date
124434 458773 129571 464846 !bounding box
28992 !epsg
0.5 !dem resolution
1 30 300 !logging display interval, interval in timesteps for 1d plots, interval in timesteps for 2d plots
0.01 0.002 !ponding water level, velocity threshold for 2d plots
0.05 0.001 !flowthreshold for 2d plots, hazard thresholds
2 !number of water levels observation locations
125023.549 463079.69
127761.3 463951.612
1 !number of horizontal cross-sections for discharge
127831.5 464373.5 127975.5 464373.5 !horizontal line points left to right
1 !number of vertical cross-sections for discharge
127369.5 464423.5 127369.5 464539.5 !vertical line points bottom to top



### Numerical settings for the computational core (settings_omp)

In the cell below we will define mathematic and machine settings for the computation core. These are generally defined once and are in principal independant of the model-run or project-area.
- <b>comp_version:</b> version of computational settings
- <b>number_cores:</b> amound of CPU cores used in computation

In [8]:
comp_version = 1
number_cores = 20

# generate file path
comp_settings_file = f"settings_omp.  {comp_version}"
comp_settings_path = project_dir.joinpath(comp_settings_file)

comp_settings = f"""4 !default max. degree of elimination
1 !preconditioner for conjugate gradient
{number_cores} !ncore, number of threads or cores
{number_cores} !m_omp - vertical slices 
1 !n_omp  - horizontal slices
8 !maximum number of non-linear iterations  # settings: max_nonlin_iterations
-8 !mineps=10**maxeps is maximum error in non-linear iterations  # settings.convergence_eps
100 1.0d-9 !default number of iteration and tolerance for solver
.0d0 0.3 !tetac, integration method: 0=standard Euler implicit,1=Carlson implicit, 2=Silecki explicit  # settings: integration_method
0 !use_of_nested_newton: 0=no, 1=yes, default=0
1.0 !pump proportion - default 1.0
1 0.045 !manning - overriden by values from files
.0d0 !dry volume
1.0d-2 !velocity threshold for friction
0.0 !1d calculation slot width
2 0.0 !thinlayer treatment option, threshold
"""

# write
comp_settings_path.write_text(comp_settings)

# print
print(comp_settings)

4 !default max. degree of elimination
1 !preconditioner for conjugate gradient
20 !ncore, number of threads or cores
20 !m_omp - vertical slices 
1 !n_omp  - horizontal slices
8 !maximum number of non-linear iterations  # settings: max_nonlin_iterations
-8 !mineps=10**maxeps is maximum error in non-linear iterations  # settings.convergence_eps
100 1.0d-9 !default number of iteration and tolerance for solver
.0d0 0.3 !tetac, integration method: 0=standard Euler implicit,1=Carlson implicit, 2=Silecki explicit  # settings: integration_method
0 !use_of_nested_newton: 0=no, 1=yes, default=0
1.0 !pump proportion - default 1.0
1 0.045 !manning - overriden by values from files
.0d0 !dry volume
1.0d-2 !velocity threshold for friction
0.0 !1d calculation slot width
2 0.0 !thinlayer treatment option, threshold



## 3. Write IO root

With ioroot we link the input-files to a directory where we store the results of the model-run. This file is picked up by the grid-generator and h2flow computation core

- <b>result_dir:</b> directory to write the results to

In [9]:
result_dir = "001_rainfall_on_grid"

# generate file path
ioroot_file = "ioroot.zzz"
ioroot_path = run_dir.joinpath(ioroot_file)

ioroot = f"""{PurePosixPath(project_dir)}#
{PurePosixPath(project_dir)}/{result_dir}/
{run_number}
"""

# write
ioroot_path.write_text(ioroot)

# print
print(ioroot)

../data/case#
../data/case/001_rainfall_on_grid/
1

